# Task B - single subject - model ?

## Notebook setup
This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- label: index of feature column to be selected to perform activity detection, between [0,6]. The default value for task B is 6;
- folder: directory name where '.mat' files are stored;
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 3
label = 6
folder = "../data/full/"
window_size = 15
stride = 5

In [2]:
import preprocessing
import models
import numpy as np
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In the following cell, we make use of some functions of Keras which have been removed, but of which the code is still available at https://github.com/keras-team/keras/commit/a56b1a55182acf061b1eb2e2c86b48193a0e88f7. These are used to evaulate the f1 score during training on batches of data: this is only an approximation though, which is the reason why they have been removed.

In [3]:
import os
if not(os.path.exists("./data")):
    os.mkdir("./data")

# One-shot classification
Here classification is performed with null class.
### Preprocessing

In [4]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=True,
                                                                                                print_info=False)

Y_train and Y_test contain the correct labels for each signals window. Y_test in particular will be used to evaluate predictions for both this (one-shot) and the two-steps models. For this reason it is here saved with a different name, to avoid having it being overwritten later.

In [5]:
Y_test_true = Y_test

### Model

In [6]:
oneshot_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

oneshot_model.compile(optimizer = Adam(lr=0.001),
                      loss = "categorical_crossentropy", 
                      metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BOS_3.hdf5', verbose=1, save_best_only=True)

### Training

In [7]:
X_train.shape

(30017, 15, 110)

In [8]:
oneshot_model.fit(x = X_train, 
                  y = to_categorical(Y_train),
                  epochs = 15,
                  batch_size = 16,
                  verbose = 1,
                  callbacks=[checkpointer],
                  validation_data=(X_test, to_categorical(Y_test)),
                  class_weight=class_weights)

Train on 30017 samples, validate on 9799 samples
Epoch 1/15
30017/30017 [==============================] - 63s 2ms/step - loss: 0.6991 - acc: 0.7850 - val_loss: 0.5491 - val_acc: 0.8186

Epoch 00001: val_loss improved from inf to 0.54907, saving model to ./data/model_BOS_3.hdf5
Epoch 2/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.4411 - acc: 0.8524 - val_loss: 0.5135 - val_acc: 0.8275

Epoch 00002: val_loss improved from 0.54907 to 0.51354, saving model to ./data/model_BOS_3.hdf5
Epoch 3/15
30017/30017 [==============================] - 57s 2ms/step - loss: 0.3425 - acc: 0.8833 - val_loss: 0.5229 - val_acc: 0.8410

Epoch 00003: val_loss did not improve
Epoch 4/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.2877 - acc: 0.9033 - val_loss: 0.5174 - val_acc: 0.8394

Epoch 00004: val_loss did not improve
Epoch 5/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.2614 - acc: 0.9100 - val_loss: 0.4814 - val_acc: 0.865

### Evaluation - passare class_weights a class report

In [9]:
Y_pred = oneshot_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.92      0.94      0.93      7668
          1       0.49      0.47      0.48        80
          2       0.67      0.72      0.69        69
          3       0.75      0.56      0.64        32
          4       0.47      0.49      0.48        91
          5       0.80      0.82      0.81        89
          6       0.37      0.35      0.36        69
          7       0.52      0.79      0.62       171
          8       0.86      0.18      0.29       136
          9       0.61      0.45      0.52        97
         10       0.48      0.42      0.45        84
         11       0.36      0.78      0.49        50
         12       0.48      0.82      0.61        72
         13       0.76      0.88      0.82       117
         14       0.49      0.45      0.47        42
         15       0.62      0.45      0.52       285
         16       0.81      0.58      0.67       575
         17       0.62      0.69      0.66   

In [10]:
oneshot_model_best = load_model('./data/model_BOS_3.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.93      0.93      0.93      7668
          1       0.44      0.54      0.48        80
          2       0.57      0.64      0.60        69
          3       0.44      0.56      0.49        32
          4       0.49      0.73      0.58        91
          5       0.59      0.96      0.73        89
          6       0.52      0.48      0.50        69
          7       0.57      0.74      0.64       171
          8       0.83      0.51      0.64       136
          9       0.37      0.47      0.42        97
         10       0.68      0.38      0.49        84
         11       0.50      0.14      0.22        50
         12       0.52      0.79      0.63        72
         13       0.64      0.68      0.66       117
         14       0.41      0.74      0.53        42
         15       0.70      0.39      0.50       285
         16       0.76      0.74      0.75       575
         17       0.83      0.76      0.80   

# Two-steps classification
## Activity detection
This model performs a binary classification.
### Preprocessing

In [11]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=True,
                                                                                                null_class=True,
                                                                                                print_info=False)

### Model

In [ ]:
detection_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BTSD_3.hdf5', verbose=1, save_best_only=True)

### Training

In [ ]:
detection_model.fit(x = X_train, 
                    y = to_categorical(Y_train), 
                    epochs = 15, 
                    batch_size = 16,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, to_categorical(Y_test)),
                    class_weight=class_weights)

Train on 30017 samples, validate on 9799 samples
Epoch 1/15
30017/30017 [==============================] - 61s 2ms/step - loss: 0.3339 - acc: 0.8598 - val_loss: 0.3233 - val_acc: 0.8612

Epoch 00001: val_loss improved from inf to 0.32334, saving model to ./data/model_BTSD_3.hdf5
Epoch 2/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.2435 - acc: 0.9019 - val_loss: 0.2952 - val_acc: 0.8701

Epoch 00002: val_loss improved from 0.32334 to 0.29517, saving model to ./data/model_BTSD_3.hdf5
Epoch 3/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.2121 - acc: 0.9116 - val_loss: 0.3197 - val_acc: 0.8702

Epoch 00003: val_loss did not improve
Epoch 4/15
30017/30017 [==============================] - 58s 2ms/step - loss: 0.1908 - acc: 0.9218 - val_loss: 0.3108 - val_acc: 0.8706

Epoch 00004: val_loss did not improve
Epoch 5/15
30017/30017 [==============================] - 57s 2ms/step - loss: 0.1721 - acc: 0.9317 - val_loss: 0.3130 - val_acc: 0.8

### Evaluation

In [ ]:
Y_pred = detection_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
detection_model_best = load_model('./data/model_BTSD_3.hdf5')

Y_pred = detection_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
Y_pred_d = Y_pred

## Activity classification

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=False,
                                                                                                print_info=False)

### Model

In [ ]:
classification_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

classification_model.compile(optimizer = Adam(lr=0.001),
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BTSC_3.hdf5', verbose=1, save_best_only=True)

### Training

In [ ]:
classification_model.fit(x = X_train,
                         y = to_categorical(Y_train), 
                         epochs = 15, 
                         batch_size = 16,
                         verbose = 1,
                         callbacks=[checkpointer],
                         validation_data=(X_test, to_categorical(Y_test)),
                         class_weight=class_weights)

### Evaluation

In [ ]:
Y_pred = classification_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
classification_model_best = load_model('./data/model_BTSC_3.hdf5')

Y_pred = classification_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

## Cascade of detection and classification
The labels that have to be used for assessment are saved in Y_test_true. The labels predicted by the detection_model are saved instead in Y_pred_d.

In [ ]:
print(Y_test_true.shape, Y_pred_d.shape)

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=True,
                                                                                                null_class=True,
                                                                                                print_info=False)

In [ ]:
mask = (Y_pred_d == 1)
X_detected = X_test[mask, :, :]
Y_pred_c = classification_model_best.predict_classes(X_detected)
Y_pred_d[mask] = Y_pred_c

In [ ]:
print(classification_report(Y_test_true, Y_pred_d))

One-shot classification instead had:

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=True,
                                                                                                print_info=False)

In [ ]:
oneshot_model_best = load_model('./data/model_BOS_3.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

# end